<a href="https://colab.research.google.com/github/byambaa1982/mongol_tweets/blob/master/tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

### Aшигласан сангууд

In [1]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import pandas as pd
import numpy as np
import regex as re
import os
import json
import time
import sys
import io
from datetime import datetime, date, time, timedelta
import pandas_gbq


# Google Cloud Platform - Using Machine Learning APIs  ).

### Security

First things first - we need to authenticate against the Google Cloud APIs.

#### Getting a Google API Credential.

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Then, when you have your key, you will enter it in this first executable cell:

In [2]:
import getpass

APIKEY = getpass.getpass()

··········


 ### Access google driver using google credentials

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
os.chdir('/content/gdrive/My Drive/fiverr/tweets')
with open("twitter_credentials.json", "r") as file:
  creds = json.load(file)
auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
api = tweepy.API(auth, wait_on_rate_limit=True)
os.chdir('/content/gdrive/My Drive/fiverr/byamba')

### Pull data using keyword and date

In [5]:
def get_tweets_csv(search_words, date_since):
  new_search = search_words + " -filter:retweets"

  tweets = tweepy.Cursor(api.search,
                              tweet_mode="extended", 
                              q=new_search,
                              lang="ru",
                              since=date_since).items(2000)

  users_locs = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.user.location
                ,tweet.favorite_count,tweet.retweet_count] for tweet in tweets]
  users_locs
  # Checks if there are coordinates attached to tweets, if so extracts them
  # users_locs['Tweet Coordinates'] = users_locs.apply(extract_coordinates,axis=1)

  tweet_text = pd.DataFrame(data=users_locs, 
                      columns=["id", "user", "tweets","date","location","favorites","retweets"])

  # tweet_text['Tweet Coordinates'] = tweet_text.apply(extract_coordinates,axis=1)
  # tweet_text.to_csv(search_words+".csv", index=False)
  return tweet_text

### Sort df by favorites

In [ ]:
df=get_tweets_csv("Хүрэлсүх", "2021-05-12")
df=df.sort_values("favorites", ascending=False)
df.head(6)
df.to_csv('khurelsukh.csv',index=False)
df1=pd.read_csv('khurelsukh.csv')
df1=df1[["user","tweets","favorites", "date"]]
df1.head(6)
df2=df1.head(100)
df2

,user,tweets,favorites,date
0,DanzanBolormaa,За Энхбат Хүрэлсүх 2ын ярилцлага үзлээ. Боловс...,274,2021-05-19 04:41:43
1,g_ganbat,АНУ-ын Мянганы сорилын сангийн УБ-ын усан ханг...,242,2021-05-18 11:37:34
2,Mandolinchin,"Нэр дэвшиж буй 3 эрхэм Энхбат ах, найз Хүрэлсү...",183,2021-05-20 07:39:09
3,tvdaria,С.Эрдэнэ 2007онд Солонгост ажиллаж бсан иргэди...,162,2021-05-17 05:25:07
4,batengineer,"Хүрэлсүх, Эрдэнэ нарын сурталчилгаа интернет о...",155,2021-05-17 02:41:41
...,...,...,...,...
95,amarsanaa,У.Хүрэлсүх: Манай Засгийн газар өрийн удирдлаг...,10,2021-05-19 03:38:10
96,Burmaa87,МАНай Хүрэлсүх би бол эгэл ардын төлөөлөл гэсэ...,10,2021-05-19 00:20:56
97,sakuna_mn,У.Хүрэлсүх: Аав ээжийгээ амьд сэрүүн байхад нь...,10,2021-05-20 14:36:07
98,ariunq,"Хүрэлсүх аав ээжийнхээ ачийг хариулсан, цуйван...",10,2021-05-21 00:12:11


In [ ]:
print('Total tweets:{}, Total favorites:{}'.format(df.shape[0], df.favorites.sum()))

Total tweets:803, Total favorites:5424


### Sentiment analysis using Google NLP API

In [19]:
lservice = build('language', 'v1beta1', developerKey=APIKEY)
def do_sentiment(quote):
  try:
    response = lservice.documents().analyzeSentiment(
    body={
      'document': {
          'type': 'PLAIN_TEXT',
          'content': quote
      }
    }).execute()
    polarity = response['documentSentiment']['polarity']
    # magnitude = response['documentSentiment']['magnitude']
    return polarity
  except:
    polarity=0
    return polarity

### Before calling NLP API, we need to translate Mongolian tweets to English
### using google translate API

In [12]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='mn', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> энэ үнэхээр амархан уу?
amazing technology -> гайхалтай технологи
wow -> хөөх


In [14]:
def translate_it(inputs):
  outputs = service.translations().list(source='mn', target='en', q=inputs).execute()
  return outputs['translations'][0]['translatedText']

### Run NLP API over each tweets to find polarity

In [ ]:
df2['english']=df2.tweets.map(lambda x:translate_it(x))
df2['polarity']=df2.english.map(lambda x:do_sentiment(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2=df2[['user','polarity','tweets','favorites']]
df2

,user,polarity,tweets,favorites
0,DanzanBolormaa,0.4,За Энхбат Хүрэлсүх 2ын ярилцлага үзлээ. Боловс...,274
1,g_ganbat,-0.8,АНУ-ын Мянганы сорилын сангийн УБ-ын усан ханг...,242
2,Mandolinchin,0.9,"Нэр дэвшиж буй 3 эрхэм Энхбат ах, найз Хүрэлсү...",183
3,tvdaria,-0.1,С.Эрдэнэ 2007онд Солонгост ажиллаж бсан иргэди...,162
4,batengineer,-1.0,"Хүрэлсүх, Эрдэнэ нарын сурталчилгаа интернет о...",155
...,...,...,...,...
95,amarsanaa,-0.1,У.Хүрэлсүх: Манай Засгийн газар өрийн удирдлаг...,10
96,Burmaa87,-1.0,МАНай Хүрэлсүх би бол эгэл ардын төлөөлөл гэсэ...,10
97,sakuna_mn,1.0,У.Хүрэлсүх: Аав ээжийгээ амьд сэрүүн байхад нь...,10
98,ariunq,-0.4,"Хүрэлсүх аав ээжийнхээ ачийг хариулсан, цуйван...",10


In [ ]:
print('Total tweets:{}, Total favorites:{}'.format(df.shape[0], df.favorites.sum()))

Total tweets:951, Total favorites:6243


In [ ]:
# table_id='tweets.enkhbat'
# project_id = 'twittersheet-275317'
# pandas_gbq.to_gbq(df, table_id, project_id=project_id ,if_exists='replace')

1it [00:02,  2.77s/it]


### Visualization

In [6]:
!ls

ananbuzar.csv	 enkhbat_v2.csv  khurelsukh.csv		     tweeters_v2.csv
anandbuzar.csv	 erdene.csv	 mongolian_top_tweeters.csv  tweets.ipynb
an.csv		 fiverr.csv	 samarsaihan.csv
battulga_v2.csv  hook.csv	 teneg_bandi.csv
enkhbat.csv	 hook_tweet.csv  top_active_tweeters.csv


In [9]:
df_k=pd.read_csv('khurelsukh.csv')

In [37]:
df_e=pd.read_csv('enkhbat_v2.csv')

In [38]:
df_e

,id,user,tweets,date,location,favorites,retweets
0,1394216127404802050,enkhbat,"Уралын өвөл. Оюутан цаг. Энхбат, Оюунаа хоёр....",2021-05-17 09:00:02,Ulaanbaatar,846,110
1,1394245887501619204,enkhbat,"Инженер Энхбат. 1990. Компьютер, сүлжээний тух...",2021-05-17 10:58:18,Ulaanbaatar,676,140
2,1394862654108278787,bambgar19,"Э.Энхбат начин ухаан орж,амь нас нь үхлээс хол...",2021-05-19 03:49:06,Mongolia,448,90
3,1393810304996364289,batengineer,Нэр дэвшигч Энхбат гуайг муулах өнцөг олох гэж...,2021-05-16 06:07:27,Mongolia,391,164
4,1392845222057418759,Sanchir_philo,"Энхбат гуайг дэмжиж буй хүмүүс сэхээтэн,оюунла...",2021-05-13 14:12:33,"Wien, Österreich",378,124
...,...,...,...,...,...,...,...
1344,1394198984135254017,tuvshee1128,@Mandukhai_ Тийм шүү.Энхбат интернэт оруулж ир...,2021-05-17 07:51:55,NaN,0,1
1345,1394197629177720834,anba_mk,Хаха өөр нэг залуу хийх л байсан. Энхбат азтай...,2021-05-17 07:46:32,NaN,0,1
1346,1394195462836539393,tulgaa68,@Mandukhai_ Өөр хэн нэгэн Энхбат бх л бсан даа...,2021-05-17 07:37:55,Mongolia,0,0
1347,1394193792094322689,idevkhtei_irgen,Д. Энхбат болон У.Хүрэлсүх гэдэг 2 хүний тухай...,2021-05-17 07:31:17,Улаанбаатар. Монгол Улс,0,0


In [39]:
df_e=df_e.head(800)

In [41]:
# df_e['english']=df_e.tweets.map(lambda x:translate_it(x))
df_e['polarity']=df_e.english.map(lambda x:do_sentiment(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
def labeling(x):
  if x >0.3:
    return 'positive'
  elif x<-0.3:
    return 'negative'
  else:
    return 'neutral'

### U.Khurelsukh

In [60]:

df_k['Expression Label'] = df_k.polarity.map(lambda x: labeling(x) )
df_k.tail()

,id,user,tweets,date,location,favorites,retweets,english,polarity,Expression Label
798,1393908343693926402,Narantuya_P,Гадуур хүмүүс Хүрэлсүхээ сонгоноо л гэж байна....,2021-05-16 12:37:01,Mongolia,0,1,Outsiders say they will choose Khurelsukhe. On...,0.2,neutral
799,1395261188313804800,BaagiiJr,"Хүрэлсүх дайныг, танкыг, бууг зохиосон хүн шүү...",2021-05-20 06:12:44,NaN,0,0,"He invented the war of touch, tanks and guns",1.0,positive
800,1395259293209808897,Bilbook0815,Монгол улсад хууль дүрэм байна уу?Суртчилгаа э...,2021-05-20 06:05:12,NaN,0,0,Is there any law in Mongolia? U.Khurelsukh sta...,-0.5,negative
801,1393916458548166664,Amarsanaa006,@Aldar51 @ariunq Хүрэлсүх ганцаараа биш шүү бу...,2021-05-16 13:09:16,Mongolia,0,0,"@ Aldar51 @ariunq Khurelsukh is not alone, eve...",-1.0,negative
802,1394295007725584384,AnjinZaVoid,@MunkhchuluunS Хүрэлсүх байгаагүй бол сайхаг б...,2021-05-17 14:13:29,ub,0,0,@MunkhchuluunS It wouldn&#39;t be nice without...,1.0,positive


In [33]:
new1 = df_k.groupby('Expression Label').count()
x = list(new1['polarity'])
y = list(new1.index)
tuple_list = list(zip(x,y))

df = pd.DataFrame(tuple_list, columns=['x','y'])

df['color'] = 'blue'
df['color'][1] = 'red'
df['color'][2] = 'green'

import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=df['x'],
                y=df['y'],
                orientation ='h',
                marker={'color': df['color']}))
fig.update_layout(
    title={
        'text': "The latest 802 tweets about the president candidate U.Khurelsukh",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
df_kh.to_csv('khurelsukh_with_sentiment.csv', index=False)

### D.Enkhbat

In [42]:
df_e['Expression Label'] = df_e.polarity.map(lambda x: labeling(x) )
df_e.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,user,tweets,date,location,favorites,retweets,english,polarity,Expression Label
795,1393052334033235969,Gereltod_Tsog,Энхбат гэдэг үгийг mute хийчихлээ 😆,2021-05-14 03:55:32,Mongolei,1,0,The word Enkhbat has been mute 😆,1.0,positive
796,1394165285347856388,Batusolu,Энхбат орж ирсэнээр Хүрэлсүх ерөнхийлөгч болох...,2021-05-17 05:38:01,хот,1,1,"With Enkhbat coming in, the chances of Khurels...",0.5,positive
797,1394909744234594308,chaimkaaaa,@bo_ulzii Аанхаа энхбат начин би сая эхнэртэй ...,2021-05-19 06:56:13,Mongolia,1,0,"@bo_ulzii Well, I just got in touch with his w...",0.1,neutral
798,1394508797205491713,Nyambaizula5,@Babrosk Энхбат байгаагүй бол интернэт байхгүй...,2021-05-18 04:23:00,"Budapest, Hungary",1,0,"@Babrosk He said that without Enkhbat, there w...",-1.0,negative
799,1393560958199549952,batjargaln,Хүрэлсүх: Би чинь муу ч гэсэн хурандаа хүн ш т...,2021-05-15 13:36:38,Somewhere in the Internet,1,0,Touch: I&#39;m a bad colonel. Erdene: You can ...,-0.4,negative


In [44]:
new1 = df_e.groupby('Expression Label').count()
x = list(new1['polarity'])
y = list(new1.index)
tuple_list = list(zip(x,y))

df = pd.DataFrame(tuple_list, columns=['x','y'])

df['color'] = 'blue'
df['color'][1] = 'red'
df['color'][2] = 'green'

import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=df['x'],
                y=df['y'],
                orientation ='h',
                marker={'color': df['color']}))
fig.update_layout(
    title={
        'text': "The latest 800 tweets about the president candidate D.Enkhbat",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
df=df_e.sort_values('date')
df=df[['Expression Label','tweets']]
df.to_csv('enkhbat_sent_analysis.csv', index=False)

In [50]:
df_e.to_csv('enkhbat_with_sentiment.csv', index=False)

### S.Erdene

In [46]:
df_er=pd.read_csv('erdene.csv')
df_er=df_er.head(800)

In [48]:
# df_er['english']=df_er.tweets.map(lambda x:translate_it(x))
df_er['polarity']=df_er.english.map(lambda x:do_sentiment(x))

In [51]:
df_er['Expression Label'] = df_er.polarity.map(lambda x: labeling(x) )
df_er.tail()

,id,user,tweets,date,location,favorites,retweets,english,polarity,Expression Label
795,1394465909578756103,timemn,Л.Оюун-Эрдэнэ: Дэлхийд гарч болох соёлын бүх к...,2021-05-18 01:32:35,Mongolia,1,0,L.Oyun-Erdene: Government will support all cul...,1.0,positive
796,1394522457403367426,odmandahk0106,Би эрдэнэ мэт ганц саналаа С.Эрдэнэд өгнө гэж ...,2021-05-18 05:17:17,"Hollands Kroon, Nederland",1,0,I decided to give my only vote to S.Erdene.,-1.0,negative
797,1394466420357046287,erdembileg2010,@AngaragDarigang Чам шиг янханд Эрдэнэ ямар га...,2021-05-18 01:34:37,NaN,1,1,"@AngaragDarigang Erdene, what happened to a pr...",-1.0,negative
798,1394522197020999681,aagii896,@myagmardorj_ Тэгээд л нөгөө хэд нь эрдэнэ гээ...,2021-05-18 05:16:15,Mongolia,1,0,@myagmardorj_ So how many of them are begging ...,-1.0,negative
799,1395027396969918469,tsdulmaa1,Зөв санаа👍👍Л.Оюун-Эрдэнэ: Төгсөөд хөдөө очиж а...,2021-05-19 14:43:44,NaN,1,0,Right idea👍👍L.Oyun-Erdene: If you go to work i...,1.0,positive


In [57]:
df=df_er.sort_values('date')
df=df[['Expression Label','tweets']]
df.to_csv('s_erdene_sent_analysis.csv', index=False)

In [53]:
new1 = df_er.groupby('Expression Label').count()
x = list(new1['polarity'])
y = list(new1.index)
tuple_list = list(zip(x,y))

df = pd.DataFrame(tuple_list, columns=['x','y'])

df['color'] = 'blue'
df['color'][1] = 'red'
df['color'][2] = 'green'

import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=df['x'],
                y=df['y'],
                orientation ='h',
                marker={'color': df['color']}))
fig.update_layout(
    title={
        'text': "The latest 800 tweets about the president candidate S.Erdene",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

